In [3]:
import json
import pandas as pd
from dataclasses import dataclass
from typing import List
import matplotlib.pyplot as plt

@dataclass
class Trade:
    timestamp: int
    buyer: str
    seller: str
    symbol: str
    currency: str
    price: float
    quantity: int

# Read entire log file
with open('./mylog.log', 'r') as f:
    content = f.read()

# --- Parse Sandbox Logs ---
sandbox_start = content.find('Sandbox logs:')
activities_start = content.find('Activities log:')
sandbox_section = content[sandbox_start + len('Sandbox logs:'):activities_start].strip()

# Each JSON object is on a new line; wrap in array brackets
# sandbox_json = '[' + ','.join(sandbox_section.split('\n')) + ']'

# sandbox_logs = json.loads(sandbox_json)

# --- Parse Activities Log ---
trades_start = content.find('Trade History:')
activities_section = content[activities_start + len('Activities log:'):trades_start].strip()

# Read CSV section into a DataFrame
from io import StringIO
activities_df = pd.read_csv(StringIO(activities_section), sep=';')

# --- Parse Trades ---
trades_section = content[trades_start + len('Trade History:'):].strip()
trades_data = json.loads(trades_section)

# Convert to Trade objects
trades: List[Trade] = [Trade(**t) for t in trades_data]

In [4]:
our_trades = [trade for trade in trades if trade.buyer == "SUBMISSION" or trade.seller == "SUBMISSION"]
# Convert to DataFrame
trades_df = pd.DataFrame([trade.__dict__ for trade in our_trades])
trades_df

,timestamp,buyer,seller,symbol,currency,price,quantity
0,6000,SUBMISSION,,CROISSANTS,SEASHELLS,4267,12
1,6000,SUBMISSION,,DJEMBES,SEASHELLS,13419,2
2,6000,SUBMISSION,,JAMS,SEASHELLS,6525,6
3,6000,,SUBMISSION,PICNIC_BASKET1,SEASHELLS,58716,2
4,6100,SUBMISSION,,CROISSANTS,SEASHELLS,4267,6
5,6100,SUBMISSION,,DJEMBES,SEASHELLS,13420,1
6,6100,SUBMISSION,,JAMS,SEASHELLS,6525,3
7,6100,,SUBMISSION,PICNIC_BASKET1,SEASHELLS,58713,1
8,6200,SUBMISSION,,CROISSANTS,SEASHELLS,4266,60
9,6200,SUBMISSION,,DJEMBES,SEASHELLS,13419,10


In [5]:
# Start Generation Here
symbols = ['PICNIC_BASKET1', 'CROISSANTS', 'JAMS', 'DJEMBES']
results = []

for symbol in symbols:
    symbol_trades = trades_df[trades_df['symbol'] == symbol]
    
    # Calculate total buys (where we are the buyer)
    total_buys = symbol_trades[symbol_trades['buyer'] == 'SUBMISSION']['quantity'].sum()
    
    # Calculate average price for buys
    buy_prices = symbol_trades[symbol_trades['buyer'] == 'SUBMISSION']['price']
    total_sells = symbol_trades[symbol_trades['seller'] == 'SUBMISSION']['quantity'].sum()
    sell_prices = symbol_trades[symbol_trades['seller'] == 'SUBMISSION']['price']
    avg_buy_price = buy_prices.mean() if not buy_prices.empty else 0
    avg_sell_price = sell_prices.mean() if not sell_prices.empty else 0
    
    results.append({
        'Symbol': symbol,
        'Total Buys': total_buys,
        'Total Sells': total_sells,
        'Average Buy Price': round(avg_buy_price, 2),
        'Average Sell Price': round(avg_sell_price, 2)
    })

# Convert results to DataFrame and display
results_df = pd.DataFrame(results)
print("\nTrade Analysis Results:")
print(results_df.to_string(index=False)) 






Trade Analysis Results:
        Symbol  Total Buys  Average Buy Price
PICNIC_BASKET1           0               0.00
    CROISSANTS         246            4266.25
          JAMS         150            6526.22
       DJEMBES          50           13418.44
